### Request data from HDFS for today

In [16]:
import json
from platformlibs.simple_data_handler import SimpleDataHandler
handler = SimpleDataHandler(sc, "mefnet-syslog", "year=2017/month=04/day=26")
rdd = handler.rdd
print rdd.count()

9242


### Parse the raw data

In [17]:
syslogs = rdd \
    .map(lambda x: dict({'timestamp': x['timestamp'], 'host': x['host_ip']}, \
                        **json.loads(x['rawdata'].decode('utf-8').replace("\n", ""))))


### Pick just the authentication failure messages

In [18]:
auth_failed = syslogs \
    .filter(lambda x: "INFO_FAILURE" in x['mnemonic'])

### Show the count of auth failure messages by host

In [19]:
countbyhost = auth_failed \
    .map(lambda x: (x['host'], 1)) \
    .combineByKey(lambda x: 1, lambda x,y: x+y, lambda x,y: x+y)
print countbyhost.collect()

[(u'10.0.2.101', 1375), (u'10.0.2.102', 1348), (u'10.0.2.103', 141)]
